In [ ]:
import pymc as pm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()

dick = pd.read_pickle('country_data.pkl')

size = 10000
exp = pm.rexponential(beta=1,size=10000)
factor = 1 + exp
#plt.hist(factor,bins=[0,1,2,3,4,5,6,7,8,9,10,11])
#factor.mean()
#np.percentile(factor,50)

swe = dick['Sweden']
mask = swe['factor'] > 1
swe_cut = swe[mask].copy()

data = swe_cut['factor']
tau = pm.Uniform('tau',lower=5,upper=15)
exp = pm.Lognormal('exp',tau, 1 / 0.1 ** 2)
obs = pm.Exponential('obs',beta=exp,value=data,observed=True)

model = pm.Model([exp,tau,obs])
map_ = pm.MAP(model)
map_.fit()

mcmc = pm.MCMC(model)

mcmc.sample(10000,5000,2)

exp_post = mcmc.trace('exp')[:]
result = pd.DataFrame({'exp_post' : exp_post})
result.describe()

posterior_data = pm.rexponential(beta=exp_post,size=exp_post.size)
posterior_data += 1
nr_samples = 1000

samples = np.random.choice(posterior_data,replace=True,size=nr_samples)
plt.hist(samples,bins=[0,1,2,3,4,5,6,7,8,9,10,11])

swe_cut.loc[:,'fake_data'] = swe_cut['confirmed'].map(lambda x :  x * np.random.choice(samples,replace=True))
   
plt.figure(figsize=(18,12))
plt.plot(range(len(swe_cut)),swe_cut['confirmed'],'x--',color='orange',label='confirmed')
plt.plot(range(len(swe_cut)),swe_cut['fake_data'],'x--',color='red',label='posterior')


starting_point = swe_cut['fake_data'][-1]

#low_samples,high_samples = np.percentile(samples,[5,95])
#samples = np.sort(samples)
#samples = samples[np.where((samples > low_samples) & (samples < high_samples))]


new_values = []
days_ahead = 10

for i in range(1,days_ahead):
    n = starting_point * np.random.choice(samples,replace=True)
    starting_point = n
    new_values.append(n)
    

last_idx = len(swe_cut)
new_index = list(range(last_idx,last_idx+len(new_values)))

new_values = pd.DataFrame(new_values,columns=['fake_data'],index=new_index)
plt.plot(new_values,'x--',color='m',label='simulated data points')
plt.legend(loc='upper left')

new_values['factor'] = new_values['fake_data'] / new_values['fake_data'].shift()
new_values.describe()